In [3]:
import numpy as np
import pandas as pd
np.random.seed(7)

In [4]:
def toNumber(inputList):
    outList = []    
    for word in inputList:
        newWord = []
        for char in word:
            newWord.append(ord(char))
        outList.append(newWord)          
    return outList

def fitToSize(word, n):
    wordFit = word[:n]
    if n > len(word):
        for i in range(n - len(word)):
            wordFit.append(0)    
    return wordFit

def fixSymbolEncode(x):
    
    if x == 8217:
        return 39
    elif x == 8220:
        return 34
    elif x == 8221:
        return 34
    elif x == 8230:
        return 46
    elif x == 8722:
        return 45
    elif x == 8211:
        return 45
    elif x == 8216:
        return 39
    elif x == 8226:
        return 46
    elif x == 8364:
        return 36
    return x

In [5]:
data = pd.read_csv('dates.csv')
Y = np.array(data.word)
X = np.array(data.label)

In [6]:
for i in range(len(X)):
    if type(X[i]) != str:
        X[i] = ''
for x in X:
    if type(x) != str:        
        print(x)

In [7]:
data = toNumber(X)

In [8]:
for i in range(len(data)):
    data[i] = fitToSize(data[i],8)

In [7]:
for i in range(len(data)):
    for j in range(len(data[i])):        
            data[i][j] = fixSymbolEncode(data[i][j])

In [8]:
maxl = 0;
maxa = []
for i in range(len(data)):
    for j in range(len(data[i])):        
           if data[i][j] > 126:
                maxl+=1                
print(maxl)

3042


In [9]:
labels = []
for y in Y:
    ans = [0,0,0]
    if y == 'NONE':
        ans[0] = 1        
    elif y == 'D_ST': 
        ans[1] = 1   
    elif y == 'D_PT':
        ans[2] = 1   
    labels.append(ans)

In [12]:
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dense
from keras.models import Model


Using Theano backend.


In [13]:
inputLayer = Input(dtype='int32', shape=(None,8,), name = 'inputLayer')
embeddingLayer = TimeDistributed(Embedding(input_dim = 9000, output_dim = 10, trainable=True, 
                                         name = 'embeddingLayer'), name='TDEmbedding')(inputLayer)
lstmLayer = TimeDistributed(LSTM(50, return_sequences=False, consume_less='gpu', 
                                name = 'lstmLayer'), name='TDlstm')(embeddingLayer)
outputLayer = TimeDistributed(Dense(output_dim=3,init='zero',activation='softmax',name='outputLayer'),
                              name='TDout')(lstmLayer)
model = Model(input=inputLayer, output=outputLayer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy','fmeasure'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
inputLayer (InputLayer)          (None, None, 8)       0                                            
____________________________________________________________________________________________________
TDEmbedding (TimeDistributed)    (None, None, 8, 10)   90000       inputLayer[0][0]                 
____________________________________________________________________________________________________
TDlstm (TimeDistributed)         (None, None, 50)      12200       TDEmbedding[0][0]                
____________________________________________________________________________________________________
TDout (TimeDistributed)          (None, None, 3)       153         TDlstm[0][0]                     
Total params: 102,353
Trainable params: 102,353
Non-trainable params: 0
___________________

In [14]:
data = np.array(data)
data.shape

(459952, 8)

In [15]:
data.shape

(459952, 8)

In [16]:
trainX = data[:459950]
trainX = trainX.reshape(45995,10,8)
trainX.shape

(45995, 10, 8)

In [17]:
trainX[:2]

array([[[ 82, 117, 115, 115, 105,  97,  39, 115],
        [ 76, 101, 110, 101, 110, 101, 114, 103],
        [105, 110,   0,   0,   0,   0,   0,   0],
        [ 99, 114, 105, 115, 105, 115,   0,   0],
        [100, 117, 101,   0,   0,   0,   0,   0],
        [116, 111,   0,   0,   0,   0,   0,   0],
        [109,  97, 110,  97, 103, 101, 109, 101],
        [105, 110, 101, 102, 102, 105,  99, 105],
        [ 77,  79,  83,  67,  79,  87,   0,   0],
        [ 77,  97, 114,   0,   0,   0,   0,   0]],

       [[ 49,  57,   0,   0,   0,   0,   0,   0],
        [ 40,  80,  82,  73,  77,  69,  41,   0],
        [ 45,  45,   0,   0,   0,   0,   0,   0],
        [ 82, 117, 115, 115, 105,  97, 110,   0],
        [112, 111, 119, 101, 114,   0,   0,   0],
        [103, 114, 105, 100,   0,   0,   0,   0],
        [ 99, 111, 109, 112,  97, 110, 121,   0],
        [ 76, 101, 110, 101, 110, 101, 114, 103],
        [ 97,   0,   0,   0,   0,   0,   0,   0],
        [117, 110, 105, 116,   0,   0,   0,   0]

In [18]:
trainY = np.array(labels[:459950])
trainY = trainY.reshape(45995,10,3)
trainY.shape

(45995, 10, 3)

In [19]:
trainY[:2]

array([[[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0]],

       [[0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]]])

In [20]:
model.fit(trainX, trainY)

Epoch 1/10
45995/45995 [==============================] - 76s - loss: 0.0524 - acc: 0.9876 - fmeasure: 0.9786         

In [21]:
answer = model.predict(trainX)

In [22]:
answer[0:20]

array([[[  9.99862552e-01,   9.77796662e-05,   3.96441646e-05],
        [  9.99752581e-01,   1.77581285e-04,   6.98329168e-05],
        [  9.99304473e-01,   4.63245990e-04,   2.32261693e-04],
        [  9.99183536e-01,   6.76111958e-04,   1.40355565e-04],
        [  9.99382198e-01,   3.97955155e-04,   2.19857247e-04],
        [  9.99512970e-01,   2.75629776e-04,   2.11413528e-04],
        [  9.99671757e-01,   2.39417321e-04,   8.88323702e-05],
        [  9.99954045e-01,   3.06043548e-05,   1.53225101e-05],
        [  9.99949276e-01,   3.66051463e-05,   1.41097689e-05],
        [  5.90118021e-02,   9.19206440e-01,   2.17817277e-02]],

       [[  7.79282689e-01,   1.40819484e-02,   2.06635341e-01],
        [  9.99764383e-01,   1.93606349e-04,   4.19953976e-05],
        [  9.99386072e-01,   4.20077180e-04,   1.93853368e-04],
        [  9.99692738e-01,   2.41288886e-04,   6.59682919e-05],
        [  9.99688447e-01,   2.34127510e-04,   7.74547079e-05],
        [  9.99481082e-01,   4.1955942

In [23]:
answer.shape

(45995, 10, 3)

In [24]:
a = answer.reshape(answer.shape[0]*answer.shape[1], answer.shape[2])

In [25]:
a.shape


(459950, 3)

In [46]:
np.argmax(a[0])

0

In [26]:
og = []
for i in range(a.shape[0]):
    if np.argmax(a[i]) == 0:
        og.append('NONE')
    elif np.argmax(a[i]) == 1:
        og.append('D_ST')
    elif np.argmax(a[i]) == 2:
        og.append('D_PT')        

In [27]:
og[:12]

['NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'D_ST',
 'NONE',
 'NONE']

In [28]:
testdf = pd.DataFrame(og, Y[:459950])
testdf.to_csv('resnet.csv')